**I. Подключение к БД**

*Импортируем библиотеку psycopg2*

In [ ]:
# !pip install psycopg2

In [ ]:
import psycopg2

*Создаём подключение к БД*

In [ ]:
conn = psycopg2.connect(host = "rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net", 
                        port = 6432,
                        database = "demo", 
                        user = "student",
                        password = "student!"
                       )

*Взаимодействие с базой осуществляется при помощи отдельного класса, cursor*

In [ ]:
cur = conn.cursor()

Пишем SQL запрос и записываем его в переменную sql

In [ ]:
sql = "SELECT aircraft_code, model FROM bookings.aircrafts_data"

Запускаем SQL запрос

In [ ]:
cur.execute(sql)

Метод description позволяет посмотреть описание данных, которые вернул метод execute

In [ ]:
cur.description

Таким способом можно извлечь список названий колонок

In [ ]:
colnames_1 = list()

for i in cur.description:
    colnames_1.append(i[0])

In [ ]:
colnames_1

Ещё один способ извлечения колонок

In [ ]:
colnames_2 = [desc[0] for desc in cur.description]
colnames_2

Напишем ещё один запрос, который забирает все данных из таблицы aircrafts_data

In [ ]:
sql = '''
SELECT *
  FROM bookings.aircrafts_data
'''

Так можно вывести содержимое переменной sql

In [ ]:
print(sql)

*Забираем результат выполнения нашего запроса и сохраняем его в переменной result*

In [ ]:
cur.execute(sql)

result = cur.fetchall()

In [ ]:
result

Данные возвращаются в виде списка (list)

In [ ]:
type(result)

*Данные можно также забирать построчно. Это удобно при последовательной обработке строк;*

In [ ]:
cur.execute(sql)

In [ ]:
cur.fetchone()

*Преобразуем результат в pandas DataFrame*

Импортируем библиотеку pandas

In [ ]:
import pandas as pd

Передадим результат запроса в DataFrame

In [ ]:
df_sql_data = pd.DataFrame(result)

In [ ]:
df_sql_data

Далее вы можете работать с результатом запроса в pandas

In [ ]:
df_sql_data[df_sql_data[0] == 'CR2']

Названия колонок выглядят неудобно. Давайте поменяем их на те, которые содержатся в таблицах БД.

Пишем наш запрос

In [ ]:
sql = '''
SELECT *
  FROM bookings.aircrafts_data
'''

In [ ]:
cur.description

Извлекаем названия колонок и сохраняем в переменную *new_column_names*

In [ ]:
new_column_names = [i[0] for i in cur.description]

In [ ]:
new_column_names

Передаем новые колонки в pandas DataFrame

In [ ]:
df_sql_data.columns = new_column_names

In [ ]:
df_sql_data.columns

In [ ]:
df_sql_data

*Вывод данных в цикле*

In [ ]:
cur.execute(sql)

for row in cur:
    print(row)

*По завершении работы с курсором его следует закрыть*

In [ ]:
cur.close()

*Чтобы постоянно не думать про cur.close(), можно выполнять транзакции в with-блоках:*

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        result = cur.fetchall()

In [ ]:
result

**II. Подключение к БД через SQLAlchemy**

Особенности SQLAlchemy:

- ORM или Object Relational Mapper (объектно-реляционное отображение) позволяет работать с базой данных с помощью объектно-ориентированного кода, не используя SQL-запросы.
- Код будет оставаться одинаковым вне зависимости от используемой БД.

In [ ]:
# !pip install sqlalchemy

In [ ]:
# !pip install sqlalchemy --upgrade

In [ ]:
# !pip uninstall sqlalchemy

In [ ]:
# !pip install sqlalchemy==1.4.51

In [ ]:
import sqlalchemy

sqlalchemy.__version__

По умолчанию SQLAlchemy работает только с базой данных SQLite без дополнительных драйверов. 

Для работы с другими базами данных необходимо установить DBAPI-совместимый драйвер в соответствии с базой данных.

Далее перечислены основные драйверы для различных БД:

- MySQL: PyMySQL, MySQL-Connector, CyMySQL, MySQL-Python 
- PostgreSQL: psycopg2, pg8000
- Microsoft SQL Server: PyODBC, pymssql
- Oracle: cx-Oracle
- Firebird: fdb, kinterbasdb

In [ ]:
from sqlalchemy import create_engine, text

Создаём функцию формирования подключения к БД

In [ ]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase demo"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

Подключаемся к БД

In [ ]:
engine = connection_yandex_cloud_demo(echo=True)

Выполнить запрос можно так:

In [ ]:
with engine.begin() as connection:
    result = connection.execute(text("SELECT * FROM bookings.aircrafts_data"))

или так:

In [ ]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM bookings.aircrafts_data"))
    connection.commit()

Результат можно вывести методом fetchall()

In [ ]:
result.fetchall()

*Запрос на чистом SQL*

In [ ]:
with engine.begin() as connection:
    select_result = connection.execute(text("SELECT * FROM bookings.airports_data")).fetchall()

In [ ]:
select_result

*Результат возвращается в виде списка. Каждый элемент списка является строкой из БД.*

In [ ]:
select_result[0]

Каждая строка в свою очередь также является списком. К элементам также можно обращаться по индексу.

In [ ]:
select_result[0][0]

In [ ]:
select_result[0][1]

In [ ]:
select_result[0][1]['en']

*Запросы с применением ORM (объектно-реляционное отображение) SQLAlchemy*

Загружаем данные о таблицах из БД demo

In [ ]:
from sqlalchemy.schema import MetaData

metadata_demo_db = MetaData(schema = 'bookings')
metadata_demo_db.reflect(bind = engine);

Выводим названия таблиц

In [ ]:
for table_name in metadata_demo_db.tables:
    print(table_name)

In [ ]:
from sqlalchemy import Table

boarding_passes = Table('boarding_passes', metadata_demo_db, autoload=True)
ticket_flights = Table('ticket_flights', metadata_demo_db, autoload=True)
flights = Table('flights', metadata_demo_db, autoload=True)
aircrafts_data = Table('aircrafts_data', metadata_demo_db, autoload=True)
airports_data = Table('airports_data', metadata_demo_db, autoload=True)
tickets = Table('tickets', metadata_demo_db, autoload=True)
bookings = Table('bookings', metadata_demo_db, autoload=True)
seats = Table('seats', metadata_demo_db, autoload=True)

In [ ]:
print(aircrafts_data.columns)

SELECT

In [ ]:
from sqlalchemy import select

In [ ]:
sql_query = select(aircrafts_data)

In [ ]:
print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

Фильтрация записей

In [ ]:
sql_query = select(aircrafts_data).where(
    aircrafts_data.c.range < 5000
)


print(sql_query)

In [ ]:
with engine.begin() as connection:
    result_with_where = connection.execute(sql_query).fetchall()

In [ ]:
result_with_where

Фильтрация по нескольким условиям

In [ ]:
sql_query = select(aircrafts_data).where(
    (aircrafts_data.c.range < 5000) &
    (aircrafts_data.c.aircraft_code == 'SU9')
)

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result_with_where = connection.execute(sql_query).fetchall()

In [ ]:
result_with_where

Логические операторы:

AND - "&"

OR - "|"

NOT - "~" 

In [ ]:
sql_query = select(aircrafts_data.c.aircraft_code, aircrafts_data.c.model).\
    where(
      (aircrafts_data.c.range > 3000 ) | 
      (aircrafts_data.c.range < 9000)
    ) 
print(sql_query, '\n')

sql_query = select(aircrafts_data).\
    where(    
      ~(aircrafts_data.c.aircraft_code == '733')
    ) 
print(sql_query, '\n')

sql_query = select(aircrafts_data).\
    where(
      ~(aircrafts_data.c.aircraft_code == '733') &
      (aircrafts_data.c.range < 3000)
    )
print(sql_query)

IS NULL

In [ ]:
sql_query = select(flights.c.flight_id, flights.c.actual_departure).\
    where(
      (flights.c.actual_departure == None)
    ) 

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

IS NOT NULL

In [ ]:
sql_query = select(flights).where(
flights.c.actual_departure != None)

print(sql_query)

IN

In [ ]:
sql_query = select(flights).\
    where(flights.c.departure_airport.in_(['DME','VKO']) )

print(sql_query,)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchone()

In [ ]:
result

NOT IN

In [ ]:
sql_query = select(flights).\
    where(flights.c.departure_airport.notin_(['DME','VKO']) )

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchone()

In [ ]:
result

BETWEEN

In [ ]:
sql_query = select(flights).\
            where(flights.c.scheduled_departure.between('2017-01-01','2018-01-01') )

print(sql_query)

LIKE

In [ ]:
sql_query = select(aircrafts_data).\
            where(aircrafts_data.c.aircraft_code.like('C%') )

print(sql_query)

Сортировка ORDER BY

In [ ]:
sql_query = select(aircrafts_data).\
            order_by(aircrafts_data.c.range).\
            where(aircrafts_data.c.range == 1200)

print(sql_query, '\n')

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

Обратная сортировка

In [ ]:
from sqlalchemy import desc

In [ ]:
sql_query = select(aircrafts_data).\
            order_by(desc(aircrafts_data.c.range))

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

LIMIT

In [ ]:
from sqlalchemy import desc

sql_query = select(aircrafts_data).\
            order_by(desc(aircrafts_data.c.range)).\
            limit(3)

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

OFFSET

In [ ]:
from sqlalchemy import desc

sql_query = select(aircrafts_data).\
            limit(3).\
            order_by(desc(aircrafts_data.c.range)).\
            offset(3)

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

LABEL (ALIAS)

In [ ]:
sql_query = select(
                aircrafts_data.c.aircraft_code
               ,aircrafts_data.c.range.label('rng')
            ).\
            order_by(desc('rng')).\
            limit(3)

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

Использование функций

In [ ]:
from sqlalchemy.sql import func

functions = [
    
    ## datetime operations
    
    func.timeofday(),
    func.localtime(),
    func.current_timestamp(),    
    func.date_part("month", func.now()),        
    func.now(),
    
    # math operations
    
    func.pow(4,2),
    func.sqrt(441),
    func.pi(),        
    func.floor(func.pi()),
    func.ceil(func.pi()),
    
    # operations with strings
    
    func.lower("ABC"),
    func.upper("abc"),
    func.length("abc")
]

for function in functions:
    select_query = select(function)
    print(select_query)

Агрегирующие функции

In [ ]:
# Примеры (!Запускать не надо):

agg_functions = [ 
    func.sum(table_name.c.column_name),
    func.avg(table_name.c.column_name),    
    func.max(table_name.c.column_name),
    func.min(table_name.c.column_name),
    func.count(table_name.c.column_name),    
]

In [ ]:
from sqlalchemy.sql import func

sql_query = select(
                func.sum(ticket_flights.c.amount).label('total_amount')
            )

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

Группировка GROUP BY

In [ ]:
sql_query = select(
                flights.c.departure_airport.label('dep_air')
               ,func.count("*")
            ).group_by('dep_air')

In [ ]:
print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result[:10]

**Объединения JOINS**

In [ ]:
print(aircrafts_data.join(seats))

In [ ]:
sql_query = select(
    aircrafts_data
    ,seats.c.seat_no
    ,seats.c.fare_conditions
).\
select_from(aircrafts_data.\
            join(seats)
           )

In [ ]:
print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result[:5]

**Оконные функции**

In [ ]:
sql_query = select(aircrafts_data.c.aircraft_code
                  ,func.sum(aircrafts_data.c.range).over(
                  partition_by = aircrafts_data.c.aircraft_code,
                  order_by = aircrafts_data.c.aircraft_code
                  )
                  )

print(sql_query)

In [ ]:
with engine.begin() as connection:
    result = connection.execute(sql_query).fetchall()

In [ ]:
result

**Загрузка данных с помощью Pandas**

*Импортируем библиотеки*

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

**Выборка данных с помощью Pandas**

In [ ]:
sql = '''
SELECT t.passenger_name
      ,f.scheduled_departure 
      ,sum(tf.amount) over(PARTITION BY t.passenger_name ORDER BY f.scheduled_departure) AS current_sum
      ,tf.amount
  FROM tickets t 
  JOIN ticket_flights tf ON t.ticket_no = tf.ticket_no
  JOIN flights f ON tf.flight_id = f.flight_id
 WHERE t.passenger_name LIKE '%OVA'
   AND EXTRACT('YEAR' FROM f.scheduled_departure) = 2017
'''

In [ ]:
print(sql)

In [ ]:
?pd.read_sql

In [ ]:
print(engine)

In [ ]:
with engine.begin() as connection:
    df = pd.read_sql(text(sql), con = connection)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df[df['passenger_name'] == 'ADELINA POPOVA'][['scheduled_departure', 'current_sum']]

Посчитаем количество перелётов для каждого месяца

In [ ]:
sql = '''
SELECT extract(YEAR FROM scheduled_departure) AS "year"
      ,extract(MONTH FROM scheduled_departure) AS "month"
      ,count(flight_id) AS count_of_flights
  FROM flights
 GROUP BY extract(YEAR FROM scheduled_departure), extract(MONTH FROM scheduled_departure)
 ORDER BY 1, 2
'''

In [ ]:
with engine.begin() as connection:
    df = pd.read_sql(sql=text(sql), con=connection)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.dtypes

Преобразуем типы двух колонок

In [ ]:
df.year = df.year.astype(int)
df.month = df.month.astype(int)

In [ ]:
df.head()

Создадим новую колонку год-месяц

In [ ]:
(df.year.astype(str) + '-' + df.month.astype(str))

In [ ]:
df['year_month'] = (df.year.astype(str) + '-' + df.month.astype(str))

In [ ]:
df.head()

Нарисуем столбчатую диаграмму кол-ва перелётов по месяцам

In [ ]:
df = df[['year_month', 'count_of_flights']]
ax = df.plot.bar(x='year_month', y='count_of_flights', title='Count of flights')

Сохраним полученные данные в excel

In [ ]:
df.to_excel('report_airport_data_2024.xlsx')